In [14]:
import cbpro
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# public client
pub_client = cbpro.PublicClient()

# authentication
f = open('/Users/jonghyunpark/turtle_trading.json','r')
data = json.load(f)

# authenticated client
locals().update(data)

auth_client = cbpro.AuthenticatedClient(API_KEY, API_SECRET, API_PASS)

pd.set_option('display.max_rows',None)

# Pseudocode for live turtle trading bot

// API call every 5 minute

IF we have a buy signal:
    
    IF we don't have position:
        
        price = X day rolling max high
        amount = total port val * 0.02 / ATR
        stop loss = buy_price - 2 ATR
    
    ELSE:
    
        IF we have money left:
        
            price = last purchase + 1 ATR
            amount = total port val * 0.02 / ATR
            stop loss = buy_price - 2 ATR
        
        ELSE: 
        
            price = same
            amount = all of the money
            stop_loss_price = (buy_price + num_pyrm/2*atr) - sl_ratio * atr
    
ELSE IF we have a sell signal:

    sell all positions

ELSE:

    Do nothing (hold positions)
    

In [15]:
# get account_info for holding coins
acc_info = {}

# coin to trade
coin = 'ETH'

for data in auth_client.get_accounts():
    currency = data['currency']
    if currency in ['USD',f'{coin}']:
        acc_info[currency] = data
        
acc_info

TypeError: string indices must be integers

In [51]:
# get price data
df = pd.DataFrame(pub_client.get_product_historic_rates(f'{coin}-USD', granularity=24*60*60),
              columns=['time','low','high','open','close','volume']).sort_values('time')
df['time'] = pd.to_datetime(df['time'],unit='s')
df.set_index('time', inplace=True)

df.head()

,low,high,open,close,volume
time,,,,,
2021-01-26,30830.00,32960.37,32260.52,32510.82,23535.838917
2021-01-27,29156.00,32584.62,32510.82,30407.13,46207.085374
2021-01-28,29902.14,34000.00,30414.11,33488.99,36007.062695
2021-01-29,31990.00,38570.13,33487.92,34262.11,72641.079750
2021-01-30,32852.86,34900.00,34262.11,34315.63,21362.151738


In [52]:
# calculate ATR and rolling high and low price
def _get_atr(df, window_atr=20, window_high=20, window_low=10):
    df['pclose'] = df['close'].shift(1)
    df['diff1'] = abs(df['high'] - df['low'])
    df['diff2'] = abs(df['pclose'] - df['high'])
    df['diff3'] = abs(df['pclose'] - df['low'])
    df['tr'] = df[['diff1', 'diff2', 'diff3']].max(axis=1)
    df['atr'] = df['tr'].rolling(window=window_atr).mean()
    df['rolling_high'] = df.high.rolling(window=window_high).max()
    df['rolling_low'] = df.low.rolling(window=window_low).min()
    return df

df = _get_atr(df, window_atr=5, window_high=5, window_low=10)

df = df.iloc[-2,:]

# df.tail()

In [53]:
df

low             57423.350000
high            59900.000000
open            58122.150000
close           59760.760000
volume           7584.652417
pclose          58122.160000
diff1            2476.650000
diff2            1777.840000
diff3             698.810000
tr               2476.650000
atr              3431.566000
rolling_high    63624.600000
rolling_low     55625.000000
Name: 2021-11-20 00:00:00, dtype: float64

In [103]:
cur_price = float(pub_client.get_product_24hr_stats(f'{coin}-USD')['last'])
cur_hold = float(acc_info[coin]['balance'])
cur_cash = float(acc_info['USD']['balance'])
cur_port_val = cur_hold * cur_price + cur_cash

print(cur_price)
print(cur_hold)
print(cur_cash)
print(cur_port_val)

59412.23
0.0001672
90.00021350858
99.93393836458


In [55]:
# parameters
n = 0.02 # pct of total port val to trade
atr = df['atr']
max_pyrm_num = 4

In [119]:
# get last buy price and get # of pyramiding
num_pyrm = -1
for i, order in enumerate(auth_client.get_fills(product_id=f'{coin}-USD')):
    if i == 0:
        last_buy_price = float(order['price'])
        last_trade_day = datetime.strptime(order['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ').day
    if order['side'] == 'sell':
        print(last_buy_price)
        print(last_trade_day)
        print(num_pyrm)
        break
    num_pyrm += 1
#     print(order)

59535.81
21
1


In [107]:
def make_buy_orders(coin, funds, size, atr):
    print(auth_client.place_market_order(product_id=f'{coin}-USD', side='buy', funds=funds))
    # stop loss at 2 * ATR
    stop_price = buy_price - 2 * atr
    auth_client.place_order(product_id=f'{coin}-USD', 
                    order_type='limit',
                    side='sell',
                    stop='loss',
                    stop_price=stop_price,
                    price=stop_price,
                    size=size)

In [120]:
cur_time = pub_client.get_product_ticker('BTC-USD')['time']
cur_time_parsed = datetime.strptime(cur_time, '%Y-%m-%dT%H:%M:%S.%fZ')
past_time = cur_time_parsed - timedelta(minutes=10)

print(cur_time_parsed)
print(past_time)

2021-11-21 03:12:06.591424
2021-11-21 03:02:06.591424


In [115]:
if (1!=1) | (2 != 2):
    print(1)

In [ ]:
# if new day started, or same day without any order, go through order process
if (cur_time_parsed.day != past_time.day) | (last_trade_day < cur_time_parsed.day):
    
    # if current price is higher than rolling high
    if cur_price > df['rolling_high']:

        # if we have no position
        if cur_hold == 0.:
            num_coins = cur_port_val * n / atr
            funds = num_coins * cur_price
            size = num_coins
            make_buy_orders(coin, funds, size, atr)

        # if we have position, and price has increased by 1 ATR, do pyramiding
        elif (cur_price > last_buy_price + atr) & (num_pyrm < max_pyrm_num):
            print(auth_client.cancel_all(product_id=f'{coin}-USD')) # cancel previous stop order
            num_coins = cur_port_val * n / atr
            funds = min(cur_price * num_coins, cur_cash) # if we don't have enough cash, we all-in
            num_coins = funds/cur_price
            size = num_coins + cur_hold
            make_buy_orders(coin, funds, size, atr)

    # if current price is lower than rolling low and we have position, sell all holdings
    elif (cur_price < df['rolling_low']) & (cur_hold > 0.):
        print(auth_client.place_market_order(product_id=f'{coin}-USD', side='sell', size=cur_hold))


In [13]:
print(auth_client.place_order(product_id=f'SOL-USD', 
                                    order_type='limit',
                                    side='sell',
                                    stop='loss',
                                    stop_price=90,
                                    price=90,
                                    size=30))

{'id': 'ea47c414-583d-4fbe-ba96-2704337210fe', 'price': '90', 'size': '30', 'product_id': 'SOL-USD', 'side': 'sell', 'stp': 'dc', 'type': 'limit', 'time_in_force': 'GTC', 'post_only': False, 'created_at': '2022-03-04T08:03:21.86504Z', 'stop': 'loss', 'stop_price': '90', 'fill_fees': '0', 'filled_size': '0', 'executed_value': '0', 'status': 'pending', 'settled': False}
